In [9]:
from utils.paths import *
from utils.dataset import *
from utils.plot_utils import *

from pkg_resources import get_platform

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output


# gps_coords = 46.08247, 7.29364
# peaks_df, observer_height = pth.get_mtn_geometry(gps_coords)

In [7]:
import importlib
importlib.reload(pth)

mdf_list, tdf_list, amdf_list  = [], [], []
for month in np.arange(1,13):
    # td = datetime.timedelta(days = interval)
    # final_date = start_date + n_intervals * td
    year = 2021
    day = 1
    date = {'year':year,'month':month,'day':day}
    date = datetime.date(year = date['year'], month = date['month'], day = date['day'])
    mdf_,tdf_,amdf_  = pth.get_data(gps_coords, observer_height, peaks_df, date)
    mdf_list.append(mdf_)
    tdf_list.append(tdf_)
    amdf_list.append(amdf_)


month = 6

mdf = mdf_list[month]
tdf = tdf_list[month]

peak_lines = dict(
                type = 'scatter',
                x = mdf.bearing,
                y = mdf.peak_angle,
                line = dict(color = 'gold'),
                mode = 'none',
                stackgroup = 'sun',
                fillcolor = 'black'
                # fill = 'tozeroy'
                )

diff_lines = dict(
                type = 'scatter',
                x = mdf.bearing,
                y = mdf.el_diff,
                line = dict(color = 'gold'),
                mode = 'none',
                stackgroup = 'sun',
                fillcolor = 'gold'
                # fill = 'tozeroy'
                )

sun_line = dict(
                type = 'scatter',
                x = mdf.loc[ mdf.daylight == 'day' ].bearing,
                y = mdf.loc[ mdf.daylight == 'day' ].elevation,
                line = dict(
                    color = 'white',
                    width = 0.5),
            )

pio.templates.default = "simple_white"

fig = go.Figure()

fig.add_traces([
    peak_lines,
    diff_lines,
    sun_line
])

annotations = []
gdf = tdf.loc[tdf.elevation > 0].groupby('time')
for hour, df in gdf:
    df = df.sort_values('date').reset_index()
    # print(df.head)
    x = float( df['azimuth' ] )
    y = float( df['elevation' ] )
    text = hour
    grad = float( df.grad )
    fig.add_trace(
        go.Scatter(
            x=[ x, x + 2 * grad],
            y=[ y, y - 2] ,
            line = dict( color="white", width=1), 
            # fill = 'toself',
            marker = None,
            mode = 'lines',
            showlegend = False
        )
    )
    # markers.append(make_marker_dict(x,y,hour))
    annotations.append( dict(
        text = str(text) + ':00',
        x = x,
        y = y,
        xanchor = 'center',
        yanchor = 'middle',
        xshift = 10 * grad,
        yshift = -20,
        showarrow = False,
        font = dict(color = 'white'),
        opacity = 1
    )    )

fig.update_layout( 
    annotations = annotations,
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, 22.5, 45.0, 67.5, 90.0, 112.5, 135.0, 157.5, 180.0, 202.5, 225.0, 247.5, 270.0, 292.5, 315, 337.5] ,
        ticktext = ['N', 'ENE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NWN'] 
    ),
    yaxis = dict(
        range = [0, 80],
        anchor = 'free',
        position = 0.5,
        visible = False
    ),
    showlegend=False,
    autosize=False,
        width=1500,
        height=800,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
)

NameError: name 'pth' is not defined

In [4]:
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

marks = { i+1:{'label':month_names[i], 'style':{'font':{'family':'Arial'} } } for i in range(12) }

In [5]:
marks

{1: {'label': 'Jan', 'style': {'font': {'family': 'Arial'}}},
 2: {'label': 'Feb', 'style': {'font': {'family': 'Arial'}}},
 3: {'label': 'Mar', 'style': {'font': {'family': 'Arial'}}},
 4: {'label': 'Apr', 'style': {'font': {'family': 'Arial'}}},
 5: {'label': 'May', 'style': {'font': {'family': 'Arial'}}},
 6: {'label': 'Jun', 'style': {'font': {'family': 'Arial'}}},
 7: {'label': 'Jul', 'style': {'font': {'family': 'Arial'}}},
 8: {'label': 'Aug', 'style': {'font': {'family': 'Arial'}}},
 9: {'label': 'Sep', 'style': {'font': {'family': 'Arial'}}},
 10: {'label': 'Oct', 'style': {'font': {'family': 'Arial'}}},
 11: {'label': 'Nov', 'style': {'font': {'family': 'Arial'}}},
 12: {'label': 'Dec', 'style': {'font': {'family': 'Arial'}}}}

In [12]:
fig = go.Figure()

slider = dict(min = 0, max = 10, marks = [1,4,5])

fig.update_layout(
    sliders = [slider]
)

fig.add_slider()

ValueError: Invalid property specified for object of type plotly.graph_objs.layout.Slider: 'min'

Did you mean "len"?

    Valid properties:
        active
            Determines which button (by index starting from 0) is
            considered active.
        activebgcolor
            Sets the background color of the slider grip while
            dragging.
        bgcolor
            Sets the background color of the slider.
        bordercolor
            Sets the color of the border enclosing the slider.
        borderwidth
            Sets the width (in px) of the border enclosing the
            slider.
        currentvalue
            :class:`plotly.graph_objects.layout.slider.Currentvalue
            ` instance or dict with compatible properties
        font
            Sets the font of the slider step labels.
        len
            Sets the length of the slider This measure excludes the
            padding of both ends. That is, the slider's length is
            this length mi